<a href="https://colab.research.google.com/github/muhammadsohaib56/Roman-Urdu-Poetry-Generator/blob/main/A1_run_updated_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# Install necessary libraries (if not already installed)
!pip install tensorflow pandas numpy

# Import required libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
import random


In [38]:
# Load Urdu poetry dataset (CSV format)
csv_file = "Roman-Urdu-Poetry.csv"  # Replace with your dataset filename

# Read CSV file (assuming poetry is in a column named 'poetry')
df = pd.read_csv(csv_file)

# Check column names and first few rows
print(df.head())

# Ensure correct column name for poetry text
poetry_column = 'Poetry'  # Update if needed

# Drop NaN values (if any)
df = df.dropna(subset=[poetry_column])

# Convert poetry to list
poems = df[poetry_column].astype(str).tolist()


   ID         Poet                                             Poetry
0   1  ahmad-faraz  aañkh se duur na ho dil se utar jā.egā \nvaqt ...
1   2  ahmad-faraz  āshiqī meñ 'mīr' jaise ḳhvāb mat dekhā karo \n...
2   3  ahmad-faraz  ab aur kyā kisī se marāsim baḌhā.eñ ham \nye b...
3   4  ahmad-faraz  ab ke ham bichhḌe to shāyad kabhī ḳhvāboñ meñ ...
4   5  ahmad-faraz  ab ke tajdīd-e-vafā kā nahīñ imkāñ jānāñ \nyaa...


In [ ]:
def clean_text(text):
    text = re.sub(r'\n', ' ', text)  # Remove newlines
    text = re.sub(r'[^ء-يA-Za-z0-9،۔ ]+', '', text)  # Keep Urdu & English characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text.lower()

# Assuming 'poetry' column contains poetry text
df['Poetry'] = df['Poetry'].astype(str).apply(clean_text)

# Convert poetry column into a list
poems = df['Poetry'].tolist()

# Display cleaned dataset
df.head()


,ID,Poet,Poetry
0,1,ahmad-faraz,aakh se duur na ho dil se utar jeg vaqt k ky h...
1,2,ahmad-faraz,shiq me mr jaise hvb mat dekh karo bvle ho jog...
2,3,ahmad-faraz,ab aur ky kis se marsim bahe ham ye bh bahut h...
3,4,ahmad-faraz,ab ke ham bichhe to shyad kabh hvbo me mile ji...
4,5,ahmad-faraz,ab ke tajddevaf k nah imk jn yaad ky tujh ko d...


In [39]:
# Define Tokenizer (without word limit to preserve vocabulary)
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(poems)

# Vocabulary size
total_words = len(tokenizer.word_index) + 1

# Convert Text to Sequences
input_sequences = tokenizer.texts_to_sequences(poems)

# Find maximum sequence length
max_sequence_length = max(len(seq) for seq in input_sequences)

# Prepare Training Data with Proper Padding
X = pad_sequences([seq[:-1] for seq in input_sequences if len(seq) > 1],
                  maxlen=max_sequence_length, padding='pre', truncating='post')

y = np.array([seq[-1] for seq in input_sequences if len(seq) > 1])

print(f"Vocabulary Size: {total_words}")
print(f"Max Sequence Length: {max_sequence_length}")


Vocabulary Size: 10524
Max Sequence Length: 566


In [45]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Bidirectional, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Ensure dataset is loaded properly
if X is None or y is None or len(X) == 0 or len(y) == 0:
    print("Warning: The dataset (X, y) is empty. Please check preprocessing.")
else:
    # Define the model architecture
    model = Sequential([
        Embedding(input_dim=total_words, output_dim=100, input_length=X.shape[1]),  # Fixed Embedding layer
        Bidirectional(GRU(256, return_sequences=True)),  # Bidirectional GRU
        Dropout(0.3),
        GRU(256),  # Additional GRU layer
        Dense(256, activation='relu'),
        Dropout(0.2),
        Dense(total_words, activation='softmax')  # Output layer
    ])

    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

    # Train the model
    history = model.fit(X, y, epochs=300, batch_size=256, verbose=1)

    # Save the trained model
    model.save("urdu_poetry_optimized.h5")


Epoch 1/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 376ms/step - accuracy: 0.0875 - loss: 9.2551
Epoch 2/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 391ms/step - accuracy: 0.1509 - loss: 8.7891
Epoch 3/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 403ms/step - accuracy: 0.1495 - loss: 5.2313
Epoch 4/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 405ms/step - accuracy: 0.1506 - loss: 4.9369
Epoch 5/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 402ms/step - accuracy: 0.1505 - loss: 4.9072
Epoch 6/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 408ms/step - accuracy: 0.1532 - loss: 4.8629
Epoch 7/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 400ms/step - accuracy: 0.1544 - loss: 4.8514
Epoch 8/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 407ms/step - accuracy: 0.1614 - loss: 4.8027
Epoch 9/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 405ms/step - accuracy: 0.1478 - loss: 4.8455
Epoch 10/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 402ms/step - accuracy: 0.1546 - loss: 4.8021
Epoch 11/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 403ms/step - accuracy: 0.1493 - loss: 4.8144
Epoch 12/300
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 402ms/step - accuracy:

In [46]:
model = tf.keras.models.load_model("urdu_poetry_optimized.h5")

In [47]:
def generate_poetry(seed_text, next_words, max_len=max_sequence_length, words_per_line=5, top_k=10):
    """Generates meaningful and structured poetry based on the given seed text."""
    seed_text = seed_text.strip()  # Clean input text
    poetry_lines = []  # List to store poetry lines
    current_line = ""  # String to build each line

    for i in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_len-1, padding='pre')

        # Predict next word
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_index = sample_from_probs(predicted_probs, top_k=top_k)  # Use top-k sampling
        predicted_word = tokenizer.index_word.get(predicted_index, '')

        # Stop if no valid word found
        if predicted_word == '':
            break

        # Append generated word
        seed_text += " " + predicted_word
        current_line += predicted_word + " "

        # Add line break after a fixed number of words
        if (i + 1) % words_per_line == 0:
            poetry_lines.append(current_line.strip())
            current_line = ""

    # Add the last line if not empty
    if current_line:
        poetry_lines.append(current_line.strip())

    return "\n".join(poetry_lines)


def sample_from_probs(probs, top_k=10):
    """Samples a word index from the top-k predicted probabilities."""
    probs = np.squeeze(probs)
    top_k_indices = np.argsort(probs)[-top_k:]  # Get top-k indices
    top_k_probs = probs[top_k_indices]
    top_k_probs = top_k_probs / np.sum(top_k_probs)  # Normalize probabilities
    sampled_index = np.random.choice(top_k_indices, p=top_k_probs)  # Sample from top-k
    return sampled_index

In [48]:
# Get user input at runtime
seed_text = input("Enter the starting word for poetry: ")
next_words = int(input("Enter the number of words to generate: "))

# Generate poetry
generated_poetry = generate_poetry(seed_text, next_words)

# Print the generated poetry
print("\nGenerated Poetry:\n")
print(generated_poetry)


Enter the starting word for poetry: dil
Enter the number of words to generate: 60

Generated Poetry:

haiñ egā hamārī haiñ hai
egā baaz ko saktā maiñ
maiñ chāhiye huuñ maiñ maiñ
hai sañvārne milā kā nahīñ
sañvāre maiñ to chale chale
maiñ milā kā nahīñ saktā
sūraj kā nahīñ saktā bachiye
alūm meñ hai abhī laayā
e tarah khatiyāñ laayā ūñ
kuchh sahī huuñ maiñ sūraj
kā nahīñ bhī nahīñ saktā
bachiye alūm meñ hai ishq


In [51]:
import gradio as gr
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the trained model
model = tf.keras.models.load_model("urdu_poetry_optimized.h5")

# Ensure the model is compiled
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

# Function to generate poetry
def generate_poetry(seed_text, next_words, max_len=50, words_per_line=5, top_k=10):
    seed_text = seed_text.strip()
    poetry_lines = []
    current_line = ""

    for i in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_len-1, padding='pre')

        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_index = sample_from_probs(predicted_probs, top_k=top_k)
        predicted_word = tokenizer.index_word.get(predicted_index, '')

        if predicted_word == '':
            break

        seed_text += " " + predicted_word
        current_line += predicted_word + " "

        if (i + 1) % words_per_line == 0:
            poetry_lines.append(current_line.strip())
            current_line = ""

    if current_line:
        poetry_lines.append(current_line.strip())

    return "\n".join(poetry_lines)

def sample_from_probs(probs, top_k=10):
    probs = np.squeeze(probs)
    top_k_indices = np.argsort(probs)[-top_k:]
    top_k_probs = probs[top_k_indices]
    top_k_probs = top_k_probs / np.sum(top_k_probs)
    return np.random.choice(top_k_indices, p=top_k_probs)

# Define Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as poetry_app:
    gr.HTML("<h1 style='text-align: center; color: #4A90E2;'>📜 Urdu Poetry Generator 🎤</h1>")

    with gr.Row():
        with gr.Column(scale=1):
            seed_text_input = gr.Textbox(label="Enter Seed Word", placeholder="Type a word to start...")
            next_words_input = gr.Number(label="Number of Words to Generate", value=10, precision=0)
            generate_button = gr.Button("Generate Poetry", variant="primary")

        with gr.Column(scale=2):
            output_area = gr.Textbox(label="Generated Poetry", lines=10, interactive=False)

    generate_button.click(fn=generate_poetry, inputs=[seed_text_input, next_words_input], outputs=output_area)

# Launch Gradio App
poetry_app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5e569204237b401b10.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
